In [8]:
from datetime import datetime
start_time0 = datetime.now()
import spacy
import numpy as np
import os
import re
import nltk
import pandas as pd
pd.set_option('display.max_colwidth', -1)
import matplotlib.pyplot as plt
import pickle
import xgboost as xgb
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import FrenchStemmer
from nltk import NaiveBayesClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, f1_score, classification_report

## Step 1 :  ML model construction 

In [15]:
train= pd.read_csv('tweets.csv',index_col=[0]).reset_index(drop=True)
train.head()

,polarity,statutnull
0,0,"Un avertissement juste: en raison de ce refus obstiné d'envisager des exemptions sans but lucratif, nous devons ne pas suivre ceux qui ne nous suivent pas."
1,4,merci!
2,0,"Réussi à écrire quelque chose en valeur! Au lit maintenant. J'espère que les musiques resteront ou bien, je pourrais pleurer."
3,4,En allant à kroger puis à la piscine. Attendant un thé à mcdon.
4,4,c'est génial


In [46]:
train = train.reset_index(drop=True)

In [47]:
train.head()

,polarity,statutnull
0,0,"- Awww, c'est nul. Tu aurais du prendre David Carr de Third Day. ;D"
1,0,"Est contrarié qu'il ne puisse pas mettre à jour son facebook par sms... et pleurera peut-être à la fin, l'école aujourd'hui aussi. Beurk!"
2,0,J'ai plongé plusieurs fois pour la balle. A réussi à en sauver 50% le reste sort de limites
3,0,Tout mon corps a des démangeaisons et comme si c'était en feu
4,0,"Non, il ne se comporte pas bien du tout. je suis en colère. pourquoi suis-je ici? Parce que je ne peux pas vous voir partout."


In [48]:
train.columns

Index(['polarity', 'statutnull'], dtype='object')

In [49]:

train = train.dropna()
train['polarity'] = train['polarity'].str.replace("0","negatif").str.replace("4","positif")
index_zero = train[(train['polarity'] != 'positif') & (train['polarity']!= 'negatif')].index
train.drop(index_zero, inplace=True)

In [50]:
train['polarity'].value_counts()
#.plot(kind='bar')

negatif    149741
positif    149615
Name: polarity, dtype: int64

In [1]:
pip install ipynb

Note: you may need to restart the kernel to use updated packages.


In [51]:
def clean_up(tweet):
    tweet = re.sub(r'http\S+|(pic.twitter\.[^\s]+)|(www\.[^\s]+)|(@\S+)|\s\s+|[^\w\s]',' ',tweet) 
    #1:http+suite/2.pic.twitter+suite/3.www.+suite/4.@+suite/5.espaces++/6.ponctuation
    tweet = tweet.lower().strip() #bdc
    tweet = word_tokenize(tweet) #tokenisation
    stop_words = stopwords.words('french') #stopwords nltk
    stop_words.append('rt') #+'rt'
    tweet = [word for word in tweet if word not in stop_words]
    tweet = [word for word in tweet if len(word)>1] #exclus mot de 1 lettre
    return ' '.join(tweet) #retour sans tokenisation, requis par spacy
def stem_spacy(tweet):
    import spacy
    nlp = spacy.load('fr_core_news_sm')
    tweet = tweet.apply(nlp)
    tweet_stem=[]
    for doc in tweet:
        tweet_stem.append([word.lemma_ for word in doc])
    return tweet_stem

In [52]:
train.statutnull = train.statutnull.astype(str)
train['original'] = train['statutnull']
train.statutnull = train.statutnull.apply(lambda s: clean_up(s))
train['tweet_stem'] = stem_spacy(train.statutnull)
train.rename(columns={'statutnull':'tweet'},inplace=True)
train.head()

,polarity,tweet,original,tweet_stem
206130,negatif,stressé mal tête stressé,Stressé ... mal à la tête ... stressé,"[stresser, mal, tête, stresser]"
227455,negatif,claire manque action,Et claire manque en action,"[clair, manquer, action]"
4963,negatif,neige avril ugh parfois déteste pittsburgh manque,"Il neige ... en avril. Ugh, parfois je déteste pittsburgh. Je me manque la","[neiger, avril, ugh, parfois, détester, pittsburgh, manquer]"
930119,positif,dernier jour lycée enfin,Dernier jour de lycée? enfin,"[dernier, jour, lycée, enfin]"
270680,negatif,nettoyer emballer affaires veux partir ici demain,Nettoyer et emballer mes affaires ... je ne veux pas partir ici demain,"[nettoyer, emballer, affairer, vouloir, partir, ici, demain]"


In [53]:
train.shape

(299356, 4)

In [54]:
train.reset_index(drop=True).head()

,polarity,tweet,original,tweet_stem
0,negatif,stressé mal tête stressé,Stressé ... mal à la tête ... stressé,"[stresser, mal, tête, stresser]"
1,negatif,claire manque action,Et claire manque en action,"[clair, manquer, action]"
2,negatif,neige avril ugh parfois déteste pittsburgh manque,"Il neige ... en avril. Ugh, parfois je déteste pittsburgh. Je me manque la","[neiger, avril, ugh, parfois, détester, pittsburgh, manquer]"
3,positif,dernier jour lycée enfin,Dernier jour de lycée? enfin,"[dernier, jour, lycée, enfin]"
4,negatif,nettoyer emballer affaires veux partir ici demain,Nettoyer et emballer mes affaires ... je ne veux pas partir ici demain,"[nettoyer, emballer, affairer, vouloir, partir, ici, demain]"


In [55]:
train['tweet_stem_join'] = train.tweet_stem.apply(lambda x: ' '.join(x)).astype(str)
train.head()

,polarity,tweet,original,tweet_stem,tweet_stem_join
206130,negatif,stressé mal tête stressé,Stressé ... mal à la tête ... stressé,"[stresser, mal, tête, stresser]",stresser mal tête stresser
227455,negatif,claire manque action,Et claire manque en action,"[clair, manquer, action]",clair manquer action
4963,negatif,neige avril ugh parfois déteste pittsburgh manque,"Il neige ... en avril. Ugh, parfois je déteste pittsburgh. Je me manque la","[neiger, avril, ugh, parfois, détester, pittsburgh, manquer]",neiger avril ugh parfois détester pittsburgh manquer
930119,positif,dernier jour lycée enfin,Dernier jour de lycée? enfin,"[dernier, jour, lycée, enfin]",dernier jour lycée enfin
270680,negatif,nettoyer emballer affaires veux partir ici demain,Nettoyer et emballer mes affaires ... je ne veux pas partir ici demain,"[nettoyer, emballer, affairer, vouloir, partir, ici, demain]",nettoyer emballer affairer vouloir partir ici demain


In [57]:
count_vectorizer = CountVectorizer(analyzer='word') 
cv = count_vectorizer.fit(train['tweet_stem_join'])
pickle.dump(cv, open("cv1.pkl", "wb"))

In [58]:
cv_ = count_vectorizer.fit_transform(train['tweet_stem_join'])
cv_.shape

(299356, 79161)

In [73]:
cv1 = pickle.load(open("cv1.pkl", 'rb'))
cv1_new = CountVectorizer(vocabulary = cv1.vocabulary_)
X_cv1 = cv1_new.transform(train['tweet_stem_join'])
X_cv1.shape

(299356, 79161)

In [60]:
end_time0 = datetime.now()
print('Temps import et clean: {}'.format(end_time0 - start_time0))

Temps import et clean: 1:42:17.555866


## SVM - à faire

## XGBoost model - non retenu

In [ ]:
start_time1 = datetime.now()
X_train_xg,X_test_xg,y_train_xg,y_test_xg = train_test_split(cv_,train['polarity'],test_size=.2,stratify=train['polarity'], random_state=42)
#grid search xgb
#grid={"max_depth":[3,6,9],'min_child_weight':[4,5,6]}
#xgbc = XGBClassifier(n_estimators=1000, nthread=6, objective= 'binary:logistic')
#xgbc_cv=GridSearchCV(xgbc,grid,cv=10)
#xgbc_cv.fit(X_train_xg,y_train_xg)
#print("tuned hpyerparameters :(best parameters) ",xgbc_cv.best_params_)
#print("accuracy :",xgbc_cv.best_score_)
#tuned hpyerparameters :(best parameters)  {'max_depth': 9, 'min_child_weight': 4}
#accuracy : 0.7395625
#model
xgbc = XGBClassifier(max_depth=9, min_child_weight=4, n_estimators=1000, nthread= 3)
xgbc.fit(X_train_xg,y_train_xg)
prediction_xgb = xgbc.predict(X_test_xg)
print(classification_report(prediction_xgb,y_test_xg))
#0.742425 count vectorizer (max_depth=6, n_estimators=1000, nthread= 3)
#0.7415 tfidf
print(accuracy_score(prediction_xgb,y_test_xg))
#0.7531 300.000 sample 21/01
end_time1 = datetime.now()
print('Temps modèle XGBoost : {}'.format(end_time1 - start_time1))
#Temps modèle XGBoost : 0:10:54.389062

## Logistic regression model - retenu

In [62]:
start_time2 = datetime.now()
X_train_logreg,X_test_logreg,y_train_logreg,y_test_logreg = train_test_split(cv_,train['polarity'] , test_size=.2,
                                                 stratify=train['polarity'], random_state=42)

In [63]:
logreg = LogisticRegression(C=1,penalty="l2")
logreg.fit(X_train_logreg,y_train_logreg)
prediction_logreg = logreg.predict(X_test_logreg)
print(classification_report(prediction_logreg,y_test_logreg))
#0.7628 count vectorizer
#0.74105 tfidf
print(accuracy_score(prediction_logreg,y_test_logreg))
end_time2 = datetime.now()
print('Temps modèle Logistic regression: {}'.format(end_time2 - start_time2))

C:\Users\aline\AppData\Roaming\Python\Python37\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


              precision    recall  f1-score   support

     negatif       0.73      0.76      0.74     28893
     positif       0.77      0.74      0.75     30979

    accuracy                           0.75     59872
   macro avg       0.75      0.75      0.75     59872
weighted avg       0.75      0.75      0.75     59872



## Naive Bayes model - non retenu

# Step 2 : applying final model on greve_twitter extraction 

In [7]:
greve_twitter=pd.read_csv('greve_twitter_final.csv',index_col=[0])
greve_twitter.head()

,polarity,date,tweet,tweet_processed,tweet_processed2,polarity2
0,negatif,NaN,Comment les travailleurs belges ont bloqué la #RetraiteParPoints #FiersDeLaGreve #reformedesretraites . #GiletsJaunes #greve31decembre #cgt #fo #sudrail #France #Belgique #BFMTV #lci #cnews,comment travailleurs belges bloqué retraiteparpoints fiersdelagreve reformedesretraites giletsjaunes greve31decembre cgt fo sudrail france belgique bfmtv lci cnews,comment travailleur belge bloquer retraiteparpoints fiersdelagreve reformedesretraites giletsjaunes greve31decembre cgt fo sudrail france belgique bfmtv lci cnews,negatif
1,negatif,NaN,#Retraites #ReformeRetraites #reformedesretraites #Macron #GiletsJaunes #greve31decembre #CGT #EdouardPhilippe #SNCF #RATP #FiersDeLaGreve 𝟮𝟳 𝗲𝗺𝗲 𝗷𝗼𝘂𝗿 de #Greve contre la Réforme...,retraites reformeretraites reformedesretraites macron giletsjaunes greve31decembre cgt edouardphilippe sncf ratp fiersdelagreve 𝟮𝟳 𝗲𝗺𝗲 𝗷𝗼𝘂𝗿 greve contre réforme,retraiter reformeretraites reformedesretraites macron giletsjaunes greve31decembre cgt edouardphilippe sncf ratp fiersdelagreve 𝟮𝟳 𝗲𝗺𝗲 𝗷𝗼𝘂𝗿 greve contrer réformer,negatif
2,negatif,NaN,#Réformedesretraites : le ton monte entre le ⁦@gouvernementFR⁩ et la #CGT - Le Parisien,réformedesretraites monte entre cgt parisien,réformedesretraites monter entrer cgt parisien,negatif
3,negatif,NaN,Apparemment ça dérange beaucoup ! ! #France #greve31decembre #reformedesretraites #GiletsJaunes #Coulommiers #écologie https://twitter.com/franck77120/status/1211370152933167105,apparemment ça dérange beaucoup france greve31decembre reformedesretraites giletsjaunes coulommiers écologie,apparemment ça déranger beaucoup france greve31decembre reformedesretraites giletsjaunes coulommiers écologie,negatif
4,negatif,NaN,"Tout à fait, un stress permanent qui se rajoutera à celui du travail, de la recherche d'un emploi, de la perte d'un emploi #reformedesretraites #RetraiteParPoints .",tout fait stress permanent rajoutera celui travail recherche emploi perte emploi reformedesretraites retraiteparpoints,tout faire stress permanent rajouter celui travail rechercher emploi perte emploi reformedesretraites retraiteparpoints,negatif


In [ ]:
X_cv2 = cv1_new.transform(greve_twitter['tweet_processed2'])
greve_twitter['polarity']=logreg.predict(X_cv2)
greve_twitter.head()